In [0]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

tf.logging.set_verbosity(tf.logging.INFO) #This way we can see the training information

In [2]:
import os
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:

%matplotlib inline
filedir = './drive/My Drive/Final/CNN_data'
filelist = os.listdir(filedir)
'''
with open(filedir + '/' + filelist[0], 'rb') as f:
  X = np.load(f)
print(X.shape)
plt.imshow(X.reshape(40,61,3)[:,:,0], cmap='hot')
plt.show()
'''

"\nwith open(filedir + '/' + filelist[0], 'rb') as f:\n  X = np.load(f)\nprint(X.shape)\nplt.imshow(X.reshape(40,61,3)[:,:,0], cmap='hot')\nplt.show()\n"

In [0]:
'''
X = np.zeros((300,7320))
Y = np.zeros(300)
for i in range(len(filelist)):
  file = filelist[i]
  if(file[0] == '0'):
    Y[i] = 0
  else:
    Y[i] = 1
  with open(filedir + '/' + file, 'rb') as f:
    data = np.load(f)
  assert (len(data) == 7320)
  X[i,:] = data
'''

"\nX = np.zeros((300,7320))\nY = np.zeros(300)\nfor i in range(len(filelist)):\n  file = filelist[i]\n  if(file[0] == '0'):\n    Y[i] = 0\n  else:\n    Y[i] = 1\n  with open(filedir + '/' + file, 'rb') as f:\n    data = np.load(f)\n  assert (len(data) == 7320)\n  X[i,:] = data\n"

In [0]:
with open(filedir + '/' + 'X', 'rb') as f:
  X = np.load(f)
with open(filedir + '/' + 'Y', 'rb') as f:
  Y = np.load(f).astype(np.int32)

In [5]:
print('X.shape: {}\nY.shape: {}'.format(X.shape, Y.shape))

X.shape: (300, 7320)
Y.shape: (300,)


In [0]:
'''
with open(filedir + '/' + 'X', 'wb') as f:
  np.save(f, X)
with open(filedir + '/' + 'Y', 'wb') as f:
  np.save(f, Y)
'''

"\nwith open(filedir + '/' + 'X', 'wb') as f:\n  np.save(f, X)\nwith open(filedir + '/' + 'Y', 'wb') as f:\n  np.save(f, Y)\n"

In [6]:
def cnn_model_fn(features, labels, mode):
  """
  Custom model function for a CNN estimator object
  """
  
  #Input preprocessing layer
  # Reshape X to 4-D tensor: [batch_size, width, height, channels]
  # spectographs are 40x183 pixels, and have one color channel
  input_layer = tf.reshape(features["x"],[-1,40,61,3]) #-1 means adjust batch size so that feature data fills the dimension of (40,61,3) before the starting the next batch element
  
  #Module 1: Extraction
  # Computes 32 new features using a 7x11 filter with ReLU activation. Filter dimensions preserve ratios of convolution 7/40~5/28, 11/(183/3)~5/28
  # Padding is added to preserve width and height during convolution.
  # The (1,1) stride makes each new feature have same dimensions as input spectrograph
  # Input Tensor Shape: [batch_size, 40, 61, 3]
  # Output Tensor Shape: [batch_size, 40, 61, 32]
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[7,11],
      strides=(1, 1),
      padding="same",
      activation=tf.nn.relu)
  # First max pooling layer with a 3x5 filter and stride of 2. Filter dimensions preserve ratios of shrinking filters 3/7~2/5, 5/11~2/5
  # The stride cuts the X's image dimensions by ceil(2)
  # Retains the newest 32 features
  # Input Tensor Shape: [batch_size, 40, 61, 32]
  # Output Tensor Shape: [batch_size, 20, 31, 32]
  pool1 = tf.layers.max_pooling2d(
      inputs=conv1,
      pool_size=[3,4],
      strides=(2,2),
      padding='same',
  )
  
  #Module 2: Extraction
  # Computes 32 new features, totaling 32+32=64 using a 7x11 filter.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 20, 31, 32]
  # Output Tensor Shape: [batch_size, 20, 31, 64]
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[7,11],
      strides=(1, 1),
      padding="same",
      activation=tf.nn.relu
  )
  # Second max pooling layer with a 3x5 filter and stride of 2
  # The stride cuts the X's image dimensions by ceil(2)
  # Retains the total 64 features
  # Input Tensor Shape: [batch_size, 20, 31, 64]
  # Output Tensor Shape: [batch_size, 10, 16, 64]
  pool2 = tf.layers.max_pooling2d(
      inputs=conv2,
      pool_size=[3,4],
      strides=(2,2),
      padding='same'
  )
  
  
  #Module 3: Prediction
  # Flatten tensor into a batch of vectors, just like our input
  # Input Tensor Shape: [batch_size, 3, 3, 3]
  # Output Tensor Shape: [batch_size, 10 * 16 * 64 = 10240]
  pool2_flattened = tf.reshape(
      pool2,
      [-1, 10 * 16 * 64]
  )
  # Densely connected layer with 1024 neurons
  # Input Tensor Shape: [batch_size, 10 * 16 * 64 = 10240]
  # Output Tensor Shape: [batch_size, 5120]
  dense = tf.layers.dense(
      inputs=pool2_flattened,
      units=5120,
      activation=tf.nn.relu)
  # Add dropout operation; 0.6 probability that element will be kept
  dropout = tf.layers.dropout(
      inputs=dense,
      rate=0.4,
      training= (mode==tf.estimator.ModeKeys.TRAIN) )
  # Input Tensor Shape: [batch_size, 5120]
  # Output Tensor Shape: [batch_size, 2]
  logits = tf.layers.dense(inputs=dropout, units=2)
  
  #Modes
  #Based on the mode, return a different value
  #Modes include: Train, Test, Predict, Eval
  
  #  Predict Mode
  predictions = {
      #Generate the logits predictions as a dictionary
      #  One key for the flattened convolution features of each input image
      "features_flat": pool2_flattened,
      #  One key for the dense features of each input image
      "features_dense": dense,
      #  One key for the predicted classes from logits
      "classes": tf.argmax(input=logits,axis=1),
      #  One key of the overal logits vector shape
      "probabilities": tf.nn.softmax(logits,name="softmax_tensor")
  }
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
  
  #  Train & Eval Mode
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
  #  Train
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step()
    )
    return tf.estimator.EstimatorSpec(
        mode=mode,
        loss=loss,
        train_op=train_op
    )
  
  #  Eval
  eval_metric_ops = {
      #Generate the evaluation metrics as a dictionary
      "accuracy": tf.metrics.accuracy(
          labels=labels,
          predictions=predictions["classes"]
      )
  }
  return tf.estimator.EstimatorSpec(
      mode=mode,
      loss=loss,
      eval_metric_ops=eval_metric_ops
  )

#log appropriately
#https://stackoverflow.com/questions/46013115/how-to-change-global-step-in-tensorflows-skcompat
config = tf.estimator.RunConfig(
  save_summary_steps=100,
  log_step_count_steps=100 #this is where we display log outputs. should be a factor of training "steps" below
)

  #create correct estimator using the custom "cnn_model_fn" defined above
digit_classifier = tf.estimator.Estimator(
  model_fn=cnn_model_fn,
  model_dir='./drive/My Drive/Final/tf_cnn_checkpoint/tf_cnn_spectograph_model',
  config=config
)
  
  #log progress while training
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
  tensors=tensors_to_log,
  every_n_iter=1000 #Only affect the long log output. Does not affect how frequently we see "steps" in output. If not divisible by number of steps, it will take precedence. IE log_iter=5, #steps=11 -> Training will last for 16steps
)

INFO:tensorflow:Using config: {'_model_dir': './drive/My Drive/Final/tf_cnn_checkpoint/tf_cnn_spectograph_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f7d271fc550>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [7]:
#prepare training input with shuffling, batching, etc
  train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X},
    y=Y,
    batch_size=50,
    num_epochs=None,
    shuffle=True
  )
  
  ##train the model by calling "estimator.train"
  digit_classifier.train(
    input_fn=train_input_fn,
    steps=10000, #Batch size = 50, total data = 300, so there are 6 steps in one epoch, we'll do 4 epochs or 32 steps over a total of 1,200 images
    hooks=None #[logging_hook]
  )

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into ./drive/My Drive/Final/tf_cnn_checkpoint/tf_cnn_spectograph_model/model.ckpt.
INFO:tensorflow:loss = 7.731426239013672, step = 0
INFO:tensorflow:global_step/sec: 9.55604
INFO:tensorflow:loss = 0.6413485407829285, step = 100 (10.466 sec)
INFO:tensorflow:global_step/sec: 9.58573
INFO:tensorflow:loss = 0.6296758651733398, step = 200 (10.436 sec)
INFO:tensorflow:global_step/sec: 9.62526
INFO:tensorflow:loss = 0.5851700305938721, step = 300 (10.390 sec)
INFO:tensorflow

In [0]:
# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, use the saver to restore variables from disk, and
# do some work with the model.
with tf.Session() as sess:
  # Restore variables from disk.
  saver.restore(sess, "/tmp/model.ckpt")
  print("Model restored.")

In [11]:

predict_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x":X},
      num_epochs=1,
      shuffle=False)
predictions = digit_classifier.predict(input_fn=predict_input_fn)
features = np.zeros((len(X), 10*16*64))
for i in range(len(X)):
  eachdata = next(predictions)
  features[i,:] = eachdata['features_flat']

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./drive/My Drive/Final/tf_cnn_checkpoint/tf_cnn_spectograph_model/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [12]:
with open('./drive/My Drive/Final/CNN_data/feature_extracted', 'wb') as f:
  np.save(f, features)


array([4.67980140e-06, 0.00000000e+00, 0.00000000e+00, ...,
       4.10164371e-03, 0.00000000e+00, 0.00000000e+00])

In [13]:
max(features[0])

1.1593384792391646